# 金、银分析

## 需要引入的库，统一添加在下一编辑框中

In [24]:
#-*-coding:utf-8 -*-
import pandas as pd
import  numpy as np
import os
from datetime import datetime,timedelta
import matplotlib.pyplot as plt
from pyecharts import Line, Overlap,Grid

# 数据所在文件夹
future代表期货数据，
spot代表现货数据，
sh代表上海市场，
london代表伦敦市场。数据维护由其他程序承担，本分析程序仅进行分析

In [3]:
gold_dir = "C:\\quanttime\\data\\gold\\sh_future\\gold.csv"
silver_dir = "C:\\quanttime\\data\\gold\\sh_future\\silver.csv"
gold_spot = "C:\\quanttime\\data\\gold\\sh_spot\\gold.csv"
silver_spot = "C:\\quanttime\\data\\gold\\sh_spot\\silver.csv"

In [21]:
gold_data = pd.read_csv(gold_dir, index_col=["date"])
silver_data = pd.read_csv(silver_dir, index_col=["date"])

gold_spot_data = pd.read_csv(gold_spot, index_col=["date"])
gold_spot_data.index = pd.to_datetime(gold_spot_data.index)
silver_spot_data = pd.read_csv(silver_spot, index_col=["date"])
silver_spot_data.index = pd.to_datetime(silver_spot_data.index)

data = pd.merge(gold_data,silver_data,left_index=True,right_index=True,suffixes=('_gold','_silver'))
spot_data = pd.merge(gold_spot_data, silver_spot_data, left_index=True, right_index=True)
#spot_data["price_gold"] = pd.to_numeric(spot_data["price_gold"],downcast='float')
#spot_data["price_silver"] = pd.to_numeric(spot_data["price_silver"],downcast='float')

print(type(gold_spot_data.index))
print(type(silver_spot_data.index))
spot_data.head(5)

<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>


,price_gold,price_silver
date,,
2006-10-30,152.41,3374.0
2006-10-31,152.52,3373.0
2006-11-01,153.18,3398.0
2006-11-02,155.42,3419.0
2006-11-03,156.94,3387.0


In [22]:
spot_data["price_silver"]=pd.to_numeric(spot_data["price_silver"],downcast="float")
print(type(spot_data.iloc[0,0]))
print(type(spot_data.iloc[0,1]))

<class 'numpy.float64'>
<class 'numpy.float32'>


In [55]:
#del nan value
data = data.dropna()
spot_data = spot_data.dropna()
#print(data.head(5))

data["compare"] = data["close_gold"]/data["close_silver"]*1000
spot_data["compare"] = spot_data["price_gold"]/spot_data["price_silver"]*1000
spot_data["price_silver"] = spot_data["price_silver"]/10 #将银价按100g为单位计价，主要是与黄金价格在同一范围
spot_data = spot_data.round(2)

In [56]:
spot_data_gold = np.array(spot_data.loc[:,["price_gold"]])
spot_data_silver = np.array(spot_data.loc[:, ["price_silver"]])
spot_data_compare = np.array(spot_data.loc[:, ["compare"]])
datatime = np.array(spot_data.index.map(str))

line_gold = Line("gold_sliver",title_top="50%")
line_compare = Line("AU/AG")

line_gold.add("gold", datatime, spot_data_gold, mark_point=["average","max","min"],legend_top="50%")
line_gold.add("silver",datatime,spot_data_silver,mark_point=["average","max","min"],legend_top="50%")
line_compare.add("AU/AG",datatime,spot_data_compare,mark_point=["average","max","min"])

grid = Grid()
grid.add(line_gold,grid_top="60%")
#grid.add(line_silver,grid_top=300)
grid.add(line_compare,grid_bottom="60%")



